In [104]:
import pandas as pd
import pythena
import psycopg2
import numpy as np
from typing import List
import random
import matplotlib.pyplot as plt
import numpy as np
import re

def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = 1000
    display.max_rows = 1000
    display.max_colwidth = 199
    display.width = 1000
    # display.precision = 2  # set as needed

set_pandas_display_options()

In [2]:
import s3fs

In [2]:
con=pythena.Athena('pdp_dev')

# 1P suggestion- chewy_amazon

In [3]:
def get1p_suggestion():
    sql='''

    select sku_uuid as uuid_a, product_segment from pdp_dev.product_warehouse where inserted_date=date('2023-09-18')
    and product_segment='pets' and source_name='chewy' limit 10
    '''
    chewy_df,id=con.execute(sql)

    sql='''
    select sku_uuid as uuid_b from pdp_dev.product_warehouse where 
    inserted_date=date('2023-09-18') and product_segment='pets' and source_name='amazon' limit 10
    '''
    amazon_df,id=con.execute(sql)

    df_1p=pd.DataFrame()
    df_1p['pair_id']=chewy_df['uuid_a']+'-'+amazon_df['uuid_b']
    df_1p['uuid_a']=chewy_df['uuid_a']
    df_1p['uuid_b']=amazon_df['uuid_b']
    df_1p['score']=1
    df_1p['score']= df_1p['score'].apply(lambda x: random.random())
    df_1p['source_type']='ML'
    df_1p['product_Segment']='pets'
    df_1p['seller_flag']='1P'
    return df_1p
df_1p=get1p_suggestion()

# 3P suggestion chewy-amazon

In [4]:
def get3p_suggestion():
    sql='''

    select sku_uuid as uuid_a, product_segment from pdp_dev.product_warehouse where inserted_date=date('2023-09-18') 
    and product_segment='pets' and source_name='chewy' limit 10
    '''
    chewy_df,id=con.execute(sql)

    sql='''
    select sku_uuid as uuid_b from pdp_dev.product_warehouse where 
    inserted_date=date('2023-09-11') and product_segment='pets' and source_name='amazon' limit 10
    '''
    amazon,id=con.execute(sql)

    df_3p=pd.DataFrame()
    df_3p['pair_id']=chewy_df['uuid_a']+'-'+amazon['uuid_b']
    df_3p['uuid_a']=chewy_df['uuid_a']
    df_3p['uuid_b']=amazon['uuid_b']
    df_3p['score']=1
    df_3p['score']= df_3p['score'].apply(lambda x: random.random())
    df_3p['source_type']='ML'
    df_3p['product_Segment']='pets'
    df_3p['seller_flag']='3P'
    return df_3p
df_3p=get3p_suggestion()

# join 1p and 3p has df

In [16]:
def get_new_suggestions():
    df_1p=get1p_suggestion()
    df_3p=get3p_suggestion()
    df=df_1p.append(df_3p)
    df['status']='un_audited'
    df['reason']=''
    return df

# take first 5 and last 5df and create match df

In [19]:
def get_existing_matches():
    df=get_new_suggestions()
    match_df=df.head(5)
    match_df=match_df.append(df.tail(5))
    match_df['BUNGEE_AUDIT_STATUS']='EXACT_MATCH'
    match_df=match_df.drop('reason',axis=1)
    match_df=match_df.drop('status',axis=1)
    return match_df

In [20]:
match_df=get_existing_matches()
match_df

C:\Users\geekd\AppData\Local\Temp\ipykernel_5524\2195134311.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df_1p.append(df_3p)
C:\Users\geekd\AppData\Local\Temp\ipykernel_5524\2913997165.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  match_df=match_df.append(df.tail(5))


,pair_id,uuid_a,uuid_b,score,source_type,product_Segment,seller_flag,BUNGEE_AUDIT_STATUS
0,963846<>chewy<>chewy-b0bzyyqbj2<>amazon<>amazon,963846<>chewy<>chewy,b0bzyyqbj2<>amazon<>amazon,0.825140,ML,pets,1P,EXACT_MATCH
1,958478<>chewy<>chewy-b0c8z2vx6h<>amazon<>amazon,958478<>chewy<>chewy,b0c8z2vx6h<>amazon<>amazon,0.914266,ML,pets,1P,EXACT_MATCH
2,956854<>chewy<>chewy-b0ch1ls52w<>amazon<>amazon,956854<>chewy<>chewy,b0ch1ls52w<>amazon<>amazon,0.324693,ML,pets,1P,EXACT_MATCH
3,963702<>chewy<>chewy-b0bh32zck8<>amazon<>amazon,963702<>chewy<>chewy,b0bh32zck8<>amazon<>amazon,0.762235,ML,pets,1P,EXACT_MATCH
4,956998<>chewy<>chewy-b0cbh8x5v3<>amazon<>amazon,956998<>chewy<>chewy,b0cbh8x5v3<>amazon<>amazon,0.733439,ML,pets,1P,EXACT_MATCH
5,957190<>chewy<>chewy-b09mq5v2v8<>amazon<>amazon,957190<>chewy<>chewy,b09mq5v2v8<>amazon<>amazon,0.092059,ML,pets,3P,EXACT_MATCH
6,959870<>chewy<>chewy-b0bpstdjjj<>amazon<>amazon,959870<>chewy<>chewy,b0bpstdjjj<>amazon<>amazon,0.651893,ML,pets,3P,EXACT_MATCH
7,959782<>chewy<>chewy-b0brp8dsy1<>amazon<>amazon,959782<>chewy<>chewy,b0brp8dsy1<>amazon<>amazon,0.042011,ML,pets,3P,EXACT_MATCH
8,958102<>chewy<>chewy-b09wn7mvlp<>amazon<>amazon,958102<>chewy<>chewy,b09wn7mvlp<>amazon<>amazon,0.037368,ML,pets,3P,EXACT_MATCH
9,958702<>chewy<>chewy-b0bz9hgknf<>amazon<>amazon,958702<>chewy<>chewy,b0bz9hgknf<>amazon<>amazon,0.427015,ML,pets,3P,EXACT_MATCH


In [21]:
match_df.to_csv('match.csv')

# suggestion Pruning based on seller type config

## prune 1psuggestion, if config is 3p , prune 3p suggestion, if config is 1p, no pruning on other cases

In [10]:
def pruneOnSellerType(df,seller_type):
    if seller_type== '1P' or seller_type=='3P':
        df=df[df['seller_flag']==seller_type]
    return df



In [11]:
def getMatch(pair_id):
    return match_df[match_df['pair_id']==pair_id]


In [12]:
def hasTypeMatch(match,match_type):
    
    if match[match['BUNGEE_AUDIT_STATUS'] == 'EXACT_MATCH'].shape[0] >0:
        return match_type in ['EXACT_MATCH','EXACT_OR_EQUIVALENT','EXACT_OVER_EQUIVALENT']
    
    if match[match['BUNGEE_AUDIT_STATUS'] == 'EQUIVALENT_MATCH'].shape[0] >0:
        return match_type in ('EQUIVALENT_MATCH','EXACT_OR_EQUIVALENT')
        
    


In [13]:
def hasSellerMatch(match,seller_type):
    
    if match[match['seller_flag']=='1P'].shape[0]>0:
        return seller_type in ['1P','1P_OR_3P','1p_OVER_3P']
   
    if match[match['seller_flag']=='3P'].shape[0]>0:
        return seller_type in ['1P_OR_3P','3P']
    
  
    

In [14]:
def execute(row):
    msg=''
    match=getMatch(row['pair_id'])
    if (hasTypeMatch(match,config['match_type']) and (hasSellerMatch(match,config['seller_type']))):
        msg=msg+'match present based on match type and seller type config'+row['pair_id']
        row['status']='pruned'
        row['reason']=msg
    return row


In [15]:
config={}
config['seller_type']='1P'
config['match_type']='EXACT_MATCH'
config['cardinality']='1'
match_df=get_existing_matches()

new_suggestions=get_new_suggestions()
procesed=new_suggestions.apply(execute,axis=1)
procesed


NameError: name 'df' is not defined

# 5-oct

In [3]:
df = pd.read_csv('s3://mdp-ut.east1/suggestion/mdp_sugg.csv')

In [38]:
df2=df.head(10)

In [39]:
df2

,Unnamed: 0,pair_id,uuid_a,uuid_b,score,source_type,product_Segment,seller_flag
0,0,963846<>chewy<>chewy-b0bzyyqbj2<>amazon<>amazon,963846<>chewy<>chewy,b0bzyyqbj2<>amazon<>amazon,0.863982,ML,pets,1P
1,1,958478<>chewy<>chewy-b0c8z2vx6h<>amazon<>amazon,958478<>chewy<>chewy,b0c8z2vx6h<>amazon<>amazon,0.269577,ML,pets,1P
2,2,956854<>chewy<>chewy-b0ch1ls52w<>amazon<>amazon,956854<>chewy<>chewy,b0ch1ls52w<>amazon<>amazon,0.713109,ML,pets,1P
3,3,963702<>chewy<>chewy-b0bh32zck8<>amazon<>amazon,963702<>chewy<>chewy,b0bh32zck8<>amazon<>amazon,0.152312,ML,pets,1P
4,4,956998<>chewy<>chewy-b0cbh8x5v3<>amazon<>amazon,956998<>chewy<>chewy,b0cbh8x5v3<>amazon<>amazon,0.492175,ML,pets,1P
5,5,957190<>chewy<>chewy-b0bzkpr9sq<>amazon<>amazon,957190<>chewy<>chewy,b0bzkpr9sq<>amazon<>amazon,0.781362,ML,pets,1P
6,6,959870<>chewy<>chewy-b0c1bvbqp2<>amazon<>amazon,959870<>chewy<>chewy,b0c1bvbqp2<>amazon<>amazon,0.209425,ML,pets,1P
7,7,959782<>chewy<>chewy-b0chbc3kz1<>amazon<>amazon,959782<>chewy<>chewy,b0chbc3kz1<>amazon<>amazon,0.468732,ML,pets,1P
8,8,958102<>chewy<>chewy-b0cd5qtld1<>amazon<>amazon,958102<>chewy<>chewy,b0cd5qtld1<>amazon<>amazon,0.183929,ML,pets,1P
9,9,958702<>chewy<>chewy-b0c9cdpt6d<>amazon<>amazon,958702<>chewy<>chewy,b0c9cdpt6d<>amazon<>amazon,0.367675,ML,pets,1P


In [44]:
df3=df2.head(5)
df3['score']=1
df3['source_type']='UPC'
df3

C:\Users\geekd\AppData\Local\Temp\ipykernel_30368\669365182.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['score']=1
C:\Users\geekd\AppData\Local\Temp\ipykernel_30368\669365182.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['source_type']='UPC'


,Unnamed: 0,pair_id,uuid_a,uuid_b,score,source_type,product_Segment,seller_flag
0,0,963846<>chewy<>chewy-b0bzyyqbj2<>amazon<>amazon,963846<>chewy<>chewy,b0bzyyqbj2<>amazon<>amazon,1,UPC,pets,1P
1,1,958478<>chewy<>chewy-b0c8z2vx6h<>amazon<>amazon,958478<>chewy<>chewy,b0c8z2vx6h<>amazon<>amazon,1,UPC,pets,1P
2,2,956854<>chewy<>chewy-b0ch1ls52w<>amazon<>amazon,956854<>chewy<>chewy,b0ch1ls52w<>amazon<>amazon,1,UPC,pets,1P
3,3,963702<>chewy<>chewy-b0bh32zck8<>amazon<>amazon,963702<>chewy<>chewy,b0bh32zck8<>amazon<>amazon,1,UPC,pets,1P
4,4,956998<>chewy<>chewy-b0cbh8x5v3<>amazon<>amazon,956998<>chewy<>chewy,b0cbh8x5v3<>amazon<>amazon,1,UPC,pets,1P


In [45]:
df4=df2.tail(5)
df4['source_type']='MPN'
df4['score']=1
df4

C:\Users\geekd\AppData\Local\Temp\ipykernel_30368\309914545.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['source_type']='MPN'
C:\Users\geekd\AppData\Local\Temp\ipykernel_30368\309914545.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['score']=1


,Unnamed: 0,pair_id,uuid_a,uuid_b,score,source_type,product_Segment,seller_flag
5,5,957190<>chewy<>chewy-b0bzkpr9sq<>amazon<>amazon,957190<>chewy<>chewy,b0bzkpr9sq<>amazon<>amazon,1,MPN,pets,1P
6,6,959870<>chewy<>chewy-b0c1bvbqp2<>amazon<>amazon,959870<>chewy<>chewy,b0c1bvbqp2<>amazon<>amazon,1,MPN,pets,1P
7,7,959782<>chewy<>chewy-b0chbc3kz1<>amazon<>amazon,959782<>chewy<>chewy,b0chbc3kz1<>amazon<>amazon,1,MPN,pets,1P
8,8,958102<>chewy<>chewy-b0cd5qtld1<>amazon<>amazon,958102<>chewy<>chewy,b0cd5qtld1<>amazon<>amazon,1,MPN,pets,1P
9,9,958702<>chewy<>chewy-b0c9cdpt6d<>amazon<>amazon,958702<>chewy<>chewy,b0c9cdpt6d<>amazon<>amazon,1,MPN,pets,1P


In [46]:
df5=df4.head(1)
df5['source_type']='UPC'

C:\Users\geekd\AppData\Local\Temp\ipykernel_30368\2961713529.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['source_type']='UPC'


In [47]:
df6=df3.head(1)
df6['source_type']='MPN'

C:\Users\geekd\AppData\Local\Temp\ipykernel_30368\94972582.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['source_type']='MPN'


In [48]:
df7=df5.append(df6)

C:\Users\geekd\AppData\Local\Temp\ipykernel_30368\3340535047.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df7=df5.append(df6)


In [31]:
df7

,Unnamed: 0,pair_id,uuid_a,uuid_b,score,source_type,product_Segment,seller_flag
15,5,957190<>chewy<>chewy-b0bzkpr9sq<>amazon<>amazon,957190<>chewy<>chewy,b0bzkpr9sq<>amazon<>amazon,1,UPC,pets,3P
0,0,963846<>chewy<>chewy-b0bzyyqbj2<>amazon<>amazon,963846<>chewy<>chewy,b0bzyyqbj2<>amazon<>amazon,1,MPN,pets,1P


In [54]:
finaldf=df2.append(df3)
finaldf=finaldf.append(df4)
finaldf=finaldf.append(df7)
finaldf

C:\Users\geekd\AppData\Local\Temp\ipykernel_30368\297538680.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finaldf=df2.append(df3)
C:\Users\geekd\AppData\Local\Temp\ipykernel_30368\297538680.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finaldf=finaldf.append(df4)
C:\Users\geekd\AppData\Local\Temp\ipykernel_30368\297538680.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finaldf=finaldf.append(df7)


,Unnamed: 0,pair_id,uuid_a,uuid_b,score,source_type,product_Segment,seller_flag
0,0,963846<>chewy<>chewy-b0bzyyqbj2<>amazon<>amazon,963846<>chewy<>chewy,b0bzyyqbj2<>amazon<>amazon,0.863982,ML,pets,1P
1,1,958478<>chewy<>chewy-b0c8z2vx6h<>amazon<>amazon,958478<>chewy<>chewy,b0c8z2vx6h<>amazon<>amazon,0.269577,ML,pets,1P
2,2,956854<>chewy<>chewy-b0ch1ls52w<>amazon<>amazon,956854<>chewy<>chewy,b0ch1ls52w<>amazon<>amazon,0.713109,ML,pets,1P
3,3,963702<>chewy<>chewy-b0bh32zck8<>amazon<>amazon,963702<>chewy<>chewy,b0bh32zck8<>amazon<>amazon,0.152312,ML,pets,1P
4,4,956998<>chewy<>chewy-b0cbh8x5v3<>amazon<>amazon,956998<>chewy<>chewy,b0cbh8x5v3<>amazon<>amazon,0.492175,ML,pets,1P
5,5,957190<>chewy<>chewy-b0bzkpr9sq<>amazon<>amazon,957190<>chewy<>chewy,b0bzkpr9sq<>amazon<>amazon,0.781362,ML,pets,1P
6,6,959870<>chewy<>chewy-b0c1bvbqp2<>amazon<>amazon,959870<>chewy<>chewy,b0c1bvbqp2<>amazon<>amazon,0.209425,ML,pets,1P
7,7,959782<>chewy<>chewy-b0chbc3kz1<>amazon<>amazon,959782<>chewy<>chewy,b0chbc3kz1<>amazon<>amazon,0.468732,ML,pets,1P
8,8,958102<>chewy<>chewy-b0cd5qtld1<>amazon<>amazon,958102<>chewy<>chewy,b0cd5qtld1<>amazon<>amazon,0.183929,ML,pets,1P
9,9,958702<>chewy<>chewy-b0c9cdpt6d<>amazon<>amazon,958702<>chewy<>chewy,b0c9cdpt6d<>amazon<>amazon,0.367675,ML,pets,1P


In [56]:
finaldf.groupby(['pair_id','source_type']).size()

pair_id                                          source_type
956854<>chewy<>chewy-b0ch1ls52w<>amazon<>amazon  ML             1
                                                 UPC            1
956998<>chewy<>chewy-b0cbh8x5v3<>amazon<>amazon  ML             1
                                                 UPC            1
957190<>chewy<>chewy-b0bzkpr9sq<>amazon<>amazon  ML             1
                                                 MPN            1
                                                 UPC            1
958102<>chewy<>chewy-b0cd5qtld1<>amazon<>amazon  ML             1
                                                 MPN            1
958478<>chewy<>chewy-b0c8z2vx6h<>amazon<>amazon  ML             1
                                                 UPC            1
958702<>chewy<>chewy-b0c9cdpt6d<>amazon<>amazon  ML             1
                                                 MPN            1
959782<>chewy<>chewy-b0chbc3kz1<>amazon<>amazon  ML             1
               

In [57]:
finaldf.to_csv('s3://mdp-ut.east1/suggestion/mdp_sugg_ag.csv')

# match work

In [96]:
mdf = pd.read_csv('s3://mdp-ut.east1/match/match.csv')

In [97]:
mdf

,Unnamed: 0,pair_id,uuid_a,uuid_b,score,source_type,product_Segment,seller_flag,BUNGEE_AUDIT_STATUS
0,0,963846<>chewy<>chewy-b0bzyyqbj2<>amazon<>amazon,963846<>chewy<>chewy,b0bzyyqbj2<>amazon<>amazon,0.825140,ML,pets,1P,EXACT_MATCH
1,1,958478<>chewy<>chewy-b0c8z2vx6h<>amazon<>amazon,958478<>chewy<>chewy,b0c8z2vx6h<>amazon<>amazon,0.914266,ML,pets,1P,EXACT_MATCH
2,2,956854<>chewy<>chewy-b0ch1ls52w<>amazon<>amazon,956854<>chewy<>chewy,b0ch1ls52w<>amazon<>amazon,0.324693,ML,pets,1P,EXACT_MATCH
3,3,963702<>chewy<>chewy-b0bh32zck8<>amazon<>amazon,963702<>chewy<>chewy,b0bh32zck8<>amazon<>amazon,0.762235,ML,pets,1P,EXACT_MATCH
4,4,956998<>chewy<>chewy-b0cbh8x5v3<>amazon<>amazon,956998<>chewy<>chewy,b0cbh8x5v3<>amazon<>amazon,0.733439,ML,pets,1P,EXACT_MATCH
5,5,957190<>chewy<>chewy-b09mq5v2v8<>amazon<>amazon,957190<>chewy<>chewy,b09mq5v2v8<>amazon<>amazon,0.092059,ML,pets,3P,EXACT_MATCH
6,6,959870<>chewy<>chewy-b0bpstdjjj<>amazon<>amazon,959870<>chewy<>chewy,b0bpstdjjj<>amazon<>amazon,0.651893,ML,pets,3P,EXACT_MATCH
7,7,959782<>chewy<>chewy-b0brp8dsy1<>amazon<>amazon,959782<>chewy<>chewy,b0brp8dsy1<>amazon<>amazon,0.042011,ML,pets,3P,EXACT_MATCH
8,8,958102<>chewy<>chewy-b09wn7mvlp<>amazon<>amazon,958102<>chewy<>chewy,b09wn7mvlp<>amazon<>amazon,0.037368,ML,pets,3P,EXACT_MATCH
9,9,958702<>chewy<>chewy-b0bz9hgknf<>amazon<>amazon,958702<>chewy<>chewy,b0bz9hgknf<>amazon<>amazon,0.427015,ML,pets,3P,EXACT_MATCH


In [98]:
mdf2=mdf.head(5)

In [99]:
mdf2['source_type']='[ML,UPC]'
mdf2['score']=mdf2['score']+1
mdf2

C:\Users\geekd\AppData\Local\Temp\ipykernel_30368\1198811154.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdf2['source_type']='[ML,UPC]'
C:\Users\geekd\AppData\Local\Temp\ipykernel_30368\1198811154.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdf2['score']=mdf2['score']+1


,Unnamed: 0,pair_id,uuid_a,uuid_b,score,source_type,product_Segment,seller_flag,BUNGEE_AUDIT_STATUS
0,0,963846<>chewy<>chewy-b0bzyyqbj2<>amazon<>amazon,963846<>chewy<>chewy,b0bzyyqbj2<>amazon<>amazon,1.825140,"[ML,UPC]",pets,1P,EXACT_MATCH
1,1,958478<>chewy<>chewy-b0c8z2vx6h<>amazon<>amazon,958478<>chewy<>chewy,b0c8z2vx6h<>amazon<>amazon,1.914266,"[ML,UPC]",pets,1P,EXACT_MATCH
2,2,956854<>chewy<>chewy-b0ch1ls52w<>amazon<>amazon,956854<>chewy<>chewy,b0ch1ls52w<>amazon<>amazon,1.324693,"[ML,UPC]",pets,1P,EXACT_MATCH
3,3,963702<>chewy<>chewy-b0bh32zck8<>amazon<>amazon,963702<>chewy<>chewy,b0bh32zck8<>amazon<>amazon,1.762235,"[ML,UPC]",pets,1P,EXACT_MATCH
4,4,956998<>chewy<>chewy-b0cbh8x5v3<>amazon<>amazon,956998<>chewy<>chewy,b0cbh8x5v3<>amazon<>amazon,1.733439,"[ML,UPC]",pets,1P,EXACT_MATCH


In [100]:
mdf3=mdf.tail(5)
mdf3=mdf3.append(mdf2)
mdf3

C:\Users\geekd\AppData\Local\Temp\ipykernel_30368\1579940545.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mdf3=mdf3.append(mdf2)


,Unnamed: 0,pair_id,uuid_a,uuid_b,score,source_type,product_Segment,seller_flag,BUNGEE_AUDIT_STATUS
5,5,957190<>chewy<>chewy-b09mq5v2v8<>amazon<>amazon,957190<>chewy<>chewy,b09mq5v2v8<>amazon<>amazon,0.092059,ML,pets,3P,EXACT_MATCH
6,6,959870<>chewy<>chewy-b0bpstdjjj<>amazon<>amazon,959870<>chewy<>chewy,b0bpstdjjj<>amazon<>amazon,0.651893,ML,pets,3P,EXACT_MATCH
7,7,959782<>chewy<>chewy-b0brp8dsy1<>amazon<>amazon,959782<>chewy<>chewy,b0brp8dsy1<>amazon<>amazon,0.042011,ML,pets,3P,EXACT_MATCH
8,8,958102<>chewy<>chewy-b09wn7mvlp<>amazon<>amazon,958102<>chewy<>chewy,b09wn7mvlp<>amazon<>amazon,0.037368,ML,pets,3P,EXACT_MATCH
9,9,958702<>chewy<>chewy-b0bz9hgknf<>amazon<>amazon,958702<>chewy<>chewy,b0bz9hgknf<>amazon<>amazon,0.427015,ML,pets,3P,EXACT_MATCH
0,0,963846<>chewy<>chewy-b0bzyyqbj2<>amazon<>amazon,963846<>chewy<>chewy,b0bzyyqbj2<>amazon<>amazon,1.825140,"[ML,UPC]",pets,1P,EXACT_MATCH
1,1,958478<>chewy<>chewy-b0c8z2vx6h<>amazon<>amazon,958478<>chewy<>chewy,b0c8z2vx6h<>amazon<>amazon,1.914266,"[ML,UPC]",pets,1P,EXACT_MATCH
2,2,956854<>chewy<>chewy-b0ch1ls52w<>amazon<>amazon,956854<>chewy<>chewy,b0ch1ls52w<>amazon<>amazon,1.324693,"[ML,UPC]",pets,1P,EXACT_MATCH
3,3,963702<>chewy<>chewy-b0bh32zck8<>amazon<>amazon,963702<>chewy<>chewy,b0bh32zck8<>amazon<>amazon,1.762235,"[ML,UPC]",pets,1P,EXACT_MATCH
4,4,956998<>chewy<>chewy-b0cbh8x5v3<>amazon<>amazon,956998<>chewy<>chewy,b0cbh8x5v3<>amazon<>amazon,1.733439,"[ML,UPC]",pets,1P,EXACT_MATCH


In [102]:
mdf3.to_csv('s3://mdp-ut.east1/match/mdp_match_agg.csv')

In [103]:
mdf3

,Unnamed: 0,pair_id,uuid_a,uuid_b,score,source_type,product_Segment,seller_flag,BUNGEE_AUDIT_STATUS
5,5,957190<>chewy<>chewy-b09mq5v2v8<>amazon<>amazon,957190<>chewy<>chewy,b09mq5v2v8<>amazon<>amazon,0.092059,ML,pets,3P,EXACT_MATCH
6,6,959870<>chewy<>chewy-b0bpstdjjj<>amazon<>amazon,959870<>chewy<>chewy,b0bpstdjjj<>amazon<>amazon,0.651893,ML,pets,3P,EXACT_MATCH
7,7,959782<>chewy<>chewy-b0brp8dsy1<>amazon<>amazon,959782<>chewy<>chewy,b0brp8dsy1<>amazon<>amazon,0.042011,ML,pets,3P,EXACT_MATCH
8,8,958102<>chewy<>chewy-b09wn7mvlp<>amazon<>amazon,958102<>chewy<>chewy,b09wn7mvlp<>amazon<>amazon,0.037368,ML,pets,3P,EXACT_MATCH
9,9,958702<>chewy<>chewy-b0bz9hgknf<>amazon<>amazon,958702<>chewy<>chewy,b0bz9hgknf<>amazon<>amazon,0.427015,ML,pets,3P,EXACT_MATCH
0,0,963846<>chewy<>chewy-b0bzyyqbj2<>amazon<>amazon,963846<>chewy<>chewy,b0bzyyqbj2<>amazon<>amazon,1.825140,"[ML,UPC]",pets,1P,EXACT_MATCH
1,1,958478<>chewy<>chewy-b0c8z2vx6h<>amazon<>amazon,958478<>chewy<>chewy,b0c8z2vx6h<>amazon<>amazon,1.914266,"[ML,UPC]",pets,1P,EXACT_MATCH
2,2,956854<>chewy<>chewy-b0ch1ls52w<>amazon<>amazon,956854<>chewy<>chewy,b0ch1ls52w<>amazon<>amazon,1.324693,"[ML,UPC]",pets,1P,EXACT_MATCH
3,3,963702<>chewy<>chewy-b0bh32zck8<>amazon<>amazon,963702<>chewy<>chewy,b0bh32zck8<>amazon<>amazon,1.762235,"[ML,UPC]",pets,1P,EXACT_MATCH
4,4,956998<>chewy<>chewy-b0cbh8x5v3<>amazon<>amazon,956998<>chewy<>chewy,b0cbh8x5v3<>amazon<>amazon,1.733439,"[ML,UPC]",pets,1P,EXACT_MATCH


# oct 9

In [195]:
md=pd.read_csv('s3://mdp-ut.east1/match/mdp_match_agg.csv')

In [196]:
md.drop('source_type',inplace=True,axis=1)

In [191]:
l=[]
l.append('ML')
l.append('UPC')
l
len(l)

2

In [212]:

def f2(r):
    r['source_type']=l
    return r
md2=md.apply(f2,axis=1)
md2.to_csv('df.csv')

In [198]:
md

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,pair_id,uuid_a,uuid_b,score,product_Segment,seller_flag,BUNGEE_AUDIT_STATUS
0,0,5,5,957190<>chewy<>chewy-b09mq5v2v8<>amazon<>amazon,957190<>chewy<>chewy,b09mq5v2v8<>amazon<>amazon,0.092059,pets,3P,EXACT_MATCH
1,1,6,6,959870<>chewy<>chewy-b0bpstdjjj<>amazon<>amazon,959870<>chewy<>chewy,b0bpstdjjj<>amazon<>amazon,0.651893,pets,3P,EXACT_MATCH
2,2,7,7,959782<>chewy<>chewy-b0brp8dsy1<>amazon<>amazon,959782<>chewy<>chewy,b0brp8dsy1<>amazon<>amazon,0.042011,pets,3P,EXACT_MATCH
3,3,8,8,958102<>chewy<>chewy-b09wn7mvlp<>amazon<>amazon,958102<>chewy<>chewy,b09wn7mvlp<>amazon<>amazon,0.037368,pets,3P,EXACT_MATCH
4,4,9,9,958702<>chewy<>chewy-b0bz9hgknf<>amazon<>amazon,958702<>chewy<>chewy,b0bz9hgknf<>amazon<>amazon,0.427015,pets,3P,EXACT_MATCH
5,5,0,0,963846<>chewy<>chewy-b0bzyyqbj2<>amazon<>amazon,963846<>chewy<>chewy,b0bzyyqbj2<>amazon<>amazon,1.825140,pets,1P,EXACT_MATCH
6,6,1,1,958478<>chewy<>chewy-b0c8z2vx6h<>amazon<>amazon,958478<>chewy<>chewy,b0c8z2vx6h<>amazon<>amazon,1.914266,pets,1P,EXACT_MATCH
7,7,2,2,956854<>chewy<>chewy-b0ch1ls52w<>amazon<>amazon,956854<>chewy<>chewy,b0ch1ls52w<>amazon<>amazon,1.324693,pets,1P,EXACT_MATCH
8,8,3,3,963702<>chewy<>chewy-b0bh32zck8<>amazon<>amazon,963702<>chewy<>chewy,b0bh32zck8<>amazon<>amazon,1.762235,pets,1P,EXACT_MATCH
9,9,4,4,956998<>chewy<>chewy-b0cbh8x5v3<>amazon<>amazon,956998<>chewy<>chewy,b0cbh8x5v3<>amazon<>amazon,1.733439,pets,1P,EXACT_MATCH
